<a href="https://colab.research.google.com/github/RamonRiosJr/RiosComputerDesigns/blob/main/Notebooks/finetune/xtts/colab_xtts_finetune_webui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Welcome to the *xtts*-finetune-webui gradio gui!

This webui is a slightly modified copy of the official webui for finetune xtts.

If you are looking for an option for normal XTTS use look here https://github.com/daswer123/xtts-webui

In [1]:
# @title 🛠️ Install requirments
#!DEBIAN_FRONTEND=noninteractive
!sudo apt-get update # && sudo apt-get -y upgrade
!sudo apt-get -y install libegl1
!sudo apt-get -y install libopengl0
!sudo apt-get -y install libxcb-cursor0
!pip install -r https://raw.githubusercontent.com/daswer123/xtts-finetune-webui/main/requirements.txt
!pip install gradio==4.44.1
!pip install fastapi==0.103.1
!pip install pydantic==2.3.0

#More requirements
!nvcc --version #check cuda version if things break
!pip uninstall torch torchvision torchaudio -y
!pip install torch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2 --index-url https://download.pytorch.org/whl/cu121


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,295 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,553 kB]
Get:13 https://developer.download.nvidia.com/compute/cuda/

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Found existing installation: torch 2.1.2
Uninstalling torch-2.1.2:
  Successfully uninstalled torch-2.1.2
Found existing installation: torchvision 0.16.2
Uninstalling torchvision-0.16.2:
  Successfully uninstalled torchvision-0.16.2
Found existing installation: torchaudio 2.1.2
Uninstalling torchaudio-2.1.2:
  Successfully uninstalled torchaudio-2.1.2
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 GB 555.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 92.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 5.1 MB/s eta 0:00:00


In [2]:
# @title 🚀 Run interface
%cd /content/
!git clone https://github.com/DrewThomasson/xtts-finetune-webui.git
%cd /content/xtts-finetune-webui
!python xtts_demo.py --share

/content
Cloning into 'xtts-finetune-webui'...
remote: Enumerating objects: 264, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 264 (delta 93), reused 67 (delta 67), pack-reused 154 (from 5)
Receiving objects: 100% (264/264), 131.32 KiB | 5.97 MiB/s, done.
Resolving deltas: 100% (130/130), done.
/content/xtts-finetune-webui
Running on local URL:  http://127.0.0.1:5003
Running on public URL: https://8c127d9b1759731333.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
Keyboard interruption in main thread... closing server.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2709, in block_thread
    time.sleep(0.1)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):


In [3]:
import shutil
import requests
import os
from tqdm import tqdm  # Progress bar library

# Define the paths
finetune_dir = '/content/xtts-finetune-webui/finetune_models/ready'  # @param {type:"string"}
dataset_dir = '/content/xtts-finetune-webui/finetune_models/dataset'  # @param {type:"string"}

# Create a temporary directory to collect both folders before zipping
temp_dir = "/content/temp_finetune_dataset"
os.makedirs(temp_dir, exist_ok=True)

# Copy both directories into the temporary directory with a progress bar
def copy_with_progress(src, dst):
    total_files = sum(len(files) for _, _, files in os.walk(src))
    with tqdm(total=total_files, desc=f"Copying {os.path.basename(src)}") as pbar:
        for root, _, files in os.walk(src):
            rel_path = os.path.relpath(root, src)
            target_path = os.path.join(dst, rel_path)
            os.makedirs(target_path, exist_ok=True)
            for file in files:
                shutil.copy(os.path.join(root, file), target_path)
                pbar.update(1)

copy_with_progress(finetune_dir, os.path.join(temp_dir, "ready"))
copy_with_progress(dataset_dir, os.path.join(temp_dir, "dataset"))

# Create a zip file of the combined directories with progress
zip_filename = "finetune_and_dataset.zip"
with tqdm(total=100, desc="Zipping files") as pbar:
    shutil.make_archive("finetune_and_dataset", 'zip', root_dir=temp_dir)
    pbar.update(100)

# Define a function to stream the upload with a progress bar
def upload_with_progress(file_path, url):
    file_size = os.path.getsize(file_path)
    with open(file_path, 'rb') as f, tqdm(
        total=file_size, unit='B', unit_scale=True, desc="Uploading"
    ) as progress:
        response = requests.post(
            url,
            files={"file": (file_path, f)},
            stream=True,
            headers={"Connection": "keep-alive"},
        )
        # Update the progress bar as chunks are sent
        for chunk in response.iter_content(chunk_size=4096):
            if chunk:
                progress.update(len(chunk))
    return response

# Upload the zip file to file.io with a progress bar
response = upload_with_progress(zip_filename, "https://file.io/?expires=1d")

# Parse the response and display the download link
if response.status_code == 200:
    download_link = response.json().get('link', 'Error: No link found.')
    print(f"Your file is ready: {download_link}")
else:
    print(f"Failed to upload: {response.status_code} - {response.text}")


Copying ready: 0it [00:00, ?it/s]
Copying dataset: 0it [00:00, ?it/s]
Zipping files: 100%|██████████| 100/100 [00:00<00:00, 167973.73it/s]
Uploading: 567kB [00:00, 1.77MB/s]


RuntimeError: The content for this response was already consumed